## Import Dependencies

In [1]:
!pip install turicreate==6.3 --quiet
!pip uninstall -y tensorflow --quiet
!pip install tensorflow-gpu --quiet

import turicreate as tc
tc.config.set_num_gpus(-1)

     |████████████████████████████████| 91.9MB 65kB/s 
     |████████████████████████████████| 86.3MB 40kB/s 
     |████████████████████████████████| 3.4MB 53.5MB/s 
     |████████████████████████████████| 327kB 53.7MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 450kB 48.0MB/s 
     |████████████████████████████████| 3.8MB 47.7MB/s 
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
     |████████████████████████████████| 320.4MB 56kB/s 
     |████████████████████████████████| 6.8MB 12.8MB/s 
     |████████████████████████████████| 460kB 34.8MB/s 
TuriCreate currently only supports using one GPU. Setting 'num_gpus' to 1.


## Download Dataset from Kaggle

In [2]:
!wget -x --load-cookies cookies.txt 'https://www.kaggle.com/c/state-farm-distracted-driver-detection/download-all' -O data.zip
!unzip -q data.zip

--2020-08-16 16:48:56--  https://www.kaggle.com/c/state-farm-distracted-driver-detection/download-all
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/5048/868335/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1597855736&Signature=FLLZ2GkHEKhnG5aVOsGdZu6deGK%2F%2FEKM6xHAmct2Kou623a2ArriZym3m0i3o%2FWzZPB%2Ftya%2BF%2FK78wJn5TFGENmdnbaanYLrNQTmlMmz8hHkpUi1CTz1fH1adY1t8whjV66jAZPian%2FQgUExCL6nspF9YtJqQ12gqewB6QXer83jwLolqXf1rmJWsuCp3xhqGP05f3sYVPWl44JHCN11CJCgfqQXlFYivWvCfROkduKPF3Qd7UllcWKBSn4qb2q4YZeFHAfDJeR97reiThHrWWr8nGdxAqAxpd1YUAkI8qx9cz%2FBqiUtGKFQoNM8WuO9rkkx35E78GHHxAqlLUaoaA%3D%3D&response-content-disposition=attachment%3B+filename%3Dstate-farm-distracted-driver-detection.zip [following]
--2020-08-16 16:48:56--  https://stora

## Train and Export Classifier

In [3]:
code = {'c0': 'Good Job',
        'c1': 'Texting Right Hand',
        'c2': 'Talking On Phone Left Hand',
        'c3': 'Texting Left Hand',
        'c4': 'Talking On Phone Left Hand',
        'c5': 'Operating Radio',
        'c6': 'Drinking',
        'c7': 'Reaching Backseat',
        'c8': 'Fixing Hair And Makeup',
        'c9': 'Talking To Passenger'}

data = tc.image_analysis.load_images('/content/imgs/train', with_path=True)
data['label'] = data['path'].apply(lambda path: code[path.split('/')[-2]])
data.save('data.sframe')

In [4]:
data = tc.SFrame('data.sframe')
train_data, test_data = data.random_split(0.8)

model = tc.image_classifier.create(train_data, 
                                   target='label', 
                                   class_weights='auto', 
                                   max_iterations=50, 
                                   batch_size=512)
predictions = model.predict(test_data)

metrics = model.evaluate(test_data)
print(metrics['accuracy'])

model.save('resnet50.model')
model.export_coreml('resnet50.mlmodel')

Instructions for updating:
non-resource variables are not supported in the long term
Download completed: /var/tmp/model_cache/resnet-50-TuriCreate-6.0.h5
Performing feature extraction on resized images...
Completed   512/17937
Completed  1024/17937
Completed  1536/17937
Completed  2048/17937
Completed  2560/17937
Completed  3072/17937
Completed  3584/17937
Completed  4096/17937
Completed  4608/17937
Completed  5120/17937
Completed  5632/17937
Completed  6144/17937
Completed  6656/17937
Completed  7168/17937
Completed  7680/17937
Completed  8192/17937
Completed  8704/17937
Completed  9216/17937
Completed  9728/17937
Completed 10240/17937
Completed 10752/17937
Completed 11264/17937
Completed 11776/17937
Completed 12288/17937
Completed 12800/17937
Completed 13312/17937
Completed 13824/17937
Completed 14336/17937
Completed 14848/17937
Completed 15360/17937
Completed 15872/17937
Completed 16384/17937
Completed 16896/17937
Completed 17408/17937
Completed 17920/17937
Completed 17937/17937
PRO

Logistic regression:

--------------------------------------------------------

Number of examples          : 17040

Number of classes           : 9

...

| 48        | 80       | 1.000000  | 134.204164   | 0.929577          | 0.901895            |

| 49        | 82       | 1.000000  | 137.270870   | 0.934624          | 0.904125            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

Performing feature extraction on resized images...
Completed   64/4487
Completed  128/4487
Completed  192/4487
Completed  256/4487
Completed  320/4487
Completed  384/4487
Completed  448/4487
Completed  512/4487
Completed  576/4487
Completed  640/4487
Completed  704/4487
Completed  768/4487
Completed  832/4487
Completed  896/4487
Completed  960/4487
Completed 1024/4487
Completed 1088/4487
Completed 1152/4487
Completed 1216/4487
Completed 1280/4487
Completed 1344/4487
Completed 1408/4487
Completed 1472/4487
Completed 1536/4487
Completed 1600/4487
Completed 1664/4487
Completed 1728/4487
Completed 1792/4487
Completed 1856/4487
Completed 1920/4487
Completed 1984/4487
Completed 2048/4487
Completed 2112/4487
Completed 2176/4487
Completed 2240/4487
Completed 2304/4487
Completed 2368/4487
Completed 2432/4487
Completed 2496/4487
Completed 2560/4487
Completed 2624/4487
Completed 2688/4487
Completed 2752/4487
Completed 2816/4487
Completed 2880/4487
Completed 2944/4487
Completed 3008/4487
Completed

Download completed: /var/tmp/model_cache/resnet-50-TuriCreate-6.0.mlmodel
